In [1]:
%%bash
pip install --upgrade pip
pip install smart_open minecart
pip install textract-trp
pip install jupyterthemes

  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2
  Created wheel for smart-open: filename=smart_open-4.0.1-py3-none-any.whl size=108249 sha256=a10519089a1b1fcb25331cade109ba78fa6410b51186211f7a3629c47a799354
  Stored in directory: /home/ec2-user/.cache/pip/wheels/8b/5e/70/42adcaea93c80417ec6accf7db1d6d02367ed02f2254cd5eef
Successfully built smart-open


In [2]:
# create a darker background template (better for my eyes)
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('chesterish')

In [3]:
import time 
import re
import os
import trp
import random 
import boto3
import minecart
import numpy as np
import pandas as pd

from smart_open import open
from sagemaker.session import Session
from io import BytesIO

%matplotlib inline

In [4]:
# initiate s3 bucket and corresponding data folder
# bucket = "ran-s3-systemic-risk"
# data_folder ="Input/X-17A-5/"

bucket = "ran-s3-systemic-risk"
data_folder ="FOCUS-OCR/SubsetTest/"

# balance_sheet_idx = 3  # the page of the report you care about

# Amazon Textract client and Sagemaker session
textract = boto3.client('textract')
s3 = boto3.client('s3')
session = Session()

# discover all of the pdfs that you want to parse
paths = np.array(session.list_s3_files(bucket, data_folder))

# AWS Asynchronous Textract Script (requesting Job)
**Content modified from Amazon AWS Textract repository (refer to [URL](https://github.com/aws-samples/amazon-textract-code-samples/blob/master/python/12-pdf-text.py) below)** 

In [5]:
def startJob(s3BucketName:str, objectName:str) -> str:
    """
    Starts a Textract job on AWS server 
    """
    response = None
    client = boto3.client('textract')
    
    # issue response to AWS to start Textract job for table analysis 
    response = client.start_document_analysis(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3BucketName,
                'Name': objectName
            }
        },
        FeatureTypes=['TABLES']
    )
    
    # return response job ID for service
    return response["JobId"]

In [6]:
def isJobComplete(jobId:str) -> str:
    """
    Tracks the completion status of the Textract job when qued
    """
    time.sleep(5)
    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    
    # check current status of AWS job (ask server every 5 seconds for data)
    while(status == "IN_PROGRESS"):
        time.sleep(5)                   # lag before reporting status
        response = client.get_document_analysis(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
    
    return status

In [7]:
def getJobResults(jobId:str) -> list:
    """
    Returns the contents of the Textract job, after completion status met
    """
    pages = []          # initialize list object to track pages

    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
    
    # iterate through the pages and append to response figure
    while(nextToken):
        response = client.get_document_analysis(JobId=jobId, NextToken=nextToken)
        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']

    return pages

In [8]:
def runJob(bucket:str, key:str):
    """
    Function designed to call an AWS Textract 
    """
    # S3 storage for files on AWS site   
    jobId = startJob(bucket, key)   # intialize Textract job 
    print("Started job with id: {}".format(jobId))

    # if job is complete from AWS return response object 
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
        
    return response

# OCR Wrapper Functions
**The scripts perform an OCR job from AWS Textract, converting tabular data into dataframes**

In [9]:
def trp2df(table:trp.Table) -> pd.DataFrame:
    """
    Function designed to convert a trp table into a dataframe
    :param table: a trp table object parsed from a pdf  
    :return: a DataFrame object housing a textracted trp table
    
    Complexity -> O(n^2)
    """
    N = len(table.rows)               # number of rows in table
    M = len(table.rows[0].cells)      # number of columns in table
    arr = [0]*N
    
    # itterate through each row within the provided table
    for row in range(N):
        
        # strip the text from the cell references to construct (N X M) matrix
        arr[row] = [table.rows[row].cells[col].text.strip() for col in range(M)]
        
    return pd.DataFrame(arr)

In [10]:
def readPDF(response:list) -> pd.DataFrame:
    """
    Function to transform AWS Textract object to a pdf
    :param response: AWS Textract response object
    """
    # in the event multiple tables detected on one page (concat them)
    catDF = []
    
    # format the Textract response type 
    doc = trp.Document(response)
    
    # iterate through document pages
    for page in doc.pages:
        # itterate through page tables
        for table in page.tables: 
            # convert trp-table into dataframe object
            df = trp2df(table)
            
            # remove columns that are completly empty (column 0 = line items)
            empty_cols = [col for col in df.columns if (df[col] == '').all()]
            df = df.drop(empty_cols, axis=1)
            
            # make sure dataframe is not empty  
            if df.values.size > 0:
                colIndex = df.columns[0]

                # check for the word "cash" in a string at the begining, ignoring case sensitivity
                assetCheck = df[colIndex].str.contains('^Cash', regex=True, flags=re.IGNORECASE)

                # check for the word "Liabilities" in a string at the end, ignoring case sensitivity
                debtCheck1 = df[colIndex].str.contains('Liabilities$|^Liabilities', 
                                                      regex=True, flags=re.IGNORECASE)
                debtCheck2 = df[colIndex].str.contains('Liability$|^Liability', 
                                                      regex=True, flags=re.IGNORECASE)

                # check if the key words have been found 
                check1 = df[assetCheck | debtCheck1 | debtCheck2].empty
                check2 = debtCheck1[debtCheck1 == True].empty
                check3 = debtCheck2[debtCheck2 == True].empty

                # if figure matched exactly, we assume this is balance sheet 
                if not check1:
                    catDF.append(df)

                    if check2 == False or check3 == False:  
                        return pd.concat(catDF)
                

## Extract Balance Sheet information

In [11]:
subsetFolder = np.array(session.list_s3_files(bucket, 'Input/SubSets/'))[1:]

In [12]:
subsetFolder

array(['Input/SubSets/1000146-02_subset.pdf',
       'Input/SubSets/1000146-03_subset.pdf',
       'Input/SubSets/1000146-04_subset.pdf', ...,
       'Input/SubSets/99947-17_subset.pdf',
       'Input/SubSets/99947-18_subset.pdf',
       'Input/SubSets/99947-19_subset.pdf'], dtype='<U35')

In [13]:
print('Total files needed to be converted {}'.format(subsetFolder.size))

Total files needed to be converted 83129


In [ ]:
# script to perform OCR (using Textract) for X-17A-5 subsets
outFolder = 'Output/BalanceSheet/'
csvDirectory = np.array(session.list_s3_files(bucket, outFolder))
errorLog = []

# iterate through X-17A-5 subsets stored in s3 
for i, key in enumerate(subsetFolder[17000:25000]):
    # baseFile name (CIK)-{Year}
    baseFile = key.split('/')[-1].split('_')[0]
    fileName = baseFile+'.csv'
    print('\nFile {}'.format(i))
    # if file is not found in directory 
    if outFolder+fileName not in csvDirectory:

        # temporary data frame object for balance sheet information
        res = runJob("ran-s3-systemic-risk", key)
        
        if res[0]['JobStatus'] != 'FAILED':
            tempDF = readPDF(res)
            print(tempDF)
            # checks for type of return 
            if type(tempDF) == pd.DataFrame:
                # writing data frame to .csv file
                tempDF.to_csv(fileName)

                # the folder used to export to
                xFolder = 'Output/BalanceSheet/'+fileName

                # save contents to AWS S3 bucket
                with open(fileName, 'rb') as data:
                    s3.put_object(Bucket=bucket, Key=xFolder, Body=data)

                # remove local file after it has been created
                os.remove(fileName)
                print('\tSaved {} file to s3 bucket'.format(baseFile+'.csv'))
            else:
                print('No Balance Sheet found in {}'.format(baseFile))
                errorLog.append(baseFile)
        else:
            print('{} could not be parsed'.format(baseFile))
            errorLog.append(baseFile)
    else:
        print('{} has been downloaded'.format(fileName))

print('==========================\nOCR is completed')


File 0
1125583-08.csv has been downloaded

File 1
1125583-09.csv has been downloaded

File 2
1125583-10.csv has been downloaded

File 3
1125583-11.csv has been downloaded

File 4
1125583-12.csv has been downloaded

File 5
1125583-13.csv has been downloaded

File 6
1125583-14.csv has been downloaded

File 7
1125583-15.csv has been downloaded

File 8
1125583-16.csv has been downloaded

File 9
1125583-17.csv has been downloaded

File 10
1125583-18.csv has been downloaded

File 11
1125583-19.csv has been downloaded

File 12
1125583-20.csv has been downloaded

File 13
1125584-02.csv has been downloaded

File 14
1125584-03.csv has been downloaded

File 15
1125584-04.csv has been downloaded

File 16
1125584-05.csv has been downloaded

File 17
1125584-06.csv has been downloaded

File 18
1125584-07.csv has been downloaded

File 19
1125584-08.csv has been downloaded

File 20
1125584-09.csv has been downloaded

File 21
1125584-10.csv has been downloaded

File 22
1125595-02.csv has been downloade

Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: FAILED
Resultset page recieved: 1
1126173-03 could not be parsed

File 94
1126173-04.csv has been downloaded

File 95
1126173-05.csv has been downloaded

File 96
1126269-02.csv has been downloaded

File 97
1126269-03.csv has been downloaded

File 98
1126269-04.csv has been downloaded

File 99
1126269-05.csv has been downloaded

File 100
1126269-06.csv has been downloaded

File 101
1126269-07.csv has been downloaded

File 102
1126269

Started job with id: 37d8333ea2ae9e76c5a54032056b547a49fbc3f16cd6c8572a30722a8292f8cf
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
None
No Balance Sheet found in 1126497-08

File 177
Started job with id: 50e86dca6b8fda2fa31b93dc35992de5dfead0fbd511eaf93bcea0ded2a3f79a
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Result

Started job with id: f1bf82fd75208c56f7cb205ea876c848a1906a189925dc364c2e8e3a0a032072
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
None
No Balance Sheet found in 1127577-07

File 320
1127577-08.csv has been downloaded

File 321
1127577-09.csv has been downloaded

File 322
Started job with id: 5d2768f5cf47896547cd4dec8f0ce80b937eb6c0e57deca286962e16dc5c57b9
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: I

Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
None
No Balance Sheet found in 1127860-11

File 376
1127860-12.csv has been downloaded

File 377
1127860-13.csv has been downloaded

File 378
1127860-14.csv has been downloaded

File 379
1127860-15.csv has been downloaded

File 380
1127860-16.csv has been downloaded

File 381
Started job with id: 840229d4b54a5fd1c4d1d53d0f866ca18e9c7e1dbfcec2261e42c6ec6fb71767
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset p

...

Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
                                                    1         2
0                                              ASSETS          
1                                                Cash  $ 11,979
2                             Due from parent company    43,197
3                                                      $ 55,176
4                LIABILITIES AND STOCKHOLDER'S EQUITY          
5                                         Liabilities       $ 0
6                                Stockholder's equity          
7   Common stock, no par value, 850 shares authori...          
8                   100 shares issued and outstanding    15,000
9                                   Retained earnings    40,176
10                         Total stockholder's equity    55,176
11                                                     $ 55,176
	Saved 1130168-02.csv file to s3 bucket

File 787
Started job with id: 5af0afd2785514ca

	Saved 1130168-07.csv file to s3 bucket

File 792
Started job with id: 86a1d8f8200ff03e81142d589f458abceb2d74a31ed455805cb4cdaa9370bcfa
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
                                                    0          1
0                                              Assets           
1                                                Cash  $ 261,167
2                                        Other assets      5,606
3                                        Total assets  $ 266,773
4                                   Liabilities Other      $ 668
5                                Stockholder's Equity           
6   Common stock, no par value, 850 shares authori...           
7                   100 shares issued and outstanding     15,000
8                          Additional paid-in-capital    275,000
9                 

	Saved 1130212-03.csv file to s3 bucket

File 799
Started job with id: 53bc9b84d2e49ed63b9eb478ad6104cd58c4e614d057e3e5a15a17cf4017a7a1
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
                                                    0          1
0                                                Cash    $ 6,797
1                          Securities owned (Note 3):           
2                               Marketable securities     52,602
3                      Due from clearing organization      1,293
4                                     Officer advance     28,995
5                                    Prepaid expenses      4,332
6                                Total current assets     94,019
7                                   Goodwill (Note 2)     58,976
8                                    Clearing deposit     12,734
9              

Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
                                                   0  \
0                                               Cash   
1                 Deposit with clearing organization   
2                                   Securities owned   
3                  Prepaid expenses and other assets   
4                                           Goodwill   
5  Furniture and equipment, less accumulated depr...   
6                                                      
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   
5                                                NaN   
6                                                NaN   

                                

Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
                                                    0          1
0                                              ASSETS           
1                                     Current assets:           
2                                                Cash   $ 17,994
3                  Deposit with clearing organization     10,018
4                                    Securities owned      2,625
5                   Prepaid expenses and other assets          -
6                                Total current assets     30,637
7                Property and equipment, net (Note 2)          -
8                                            Goodwill     58,976
9                                            Deposits      1,400
10                                       Total assets   $ 91,013
11               LIAB

In [ ]:
np.array(errorLog)